In [3]:
import pandas as pd
import numpy as np
#import statsmodels as stats
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import scipy 
#from statsmodels.tsa.stattools  import   grangercausalitytests

print('pandas: %s' % pd.__version__)
print('numpy: %s' % np.__version__)
#print('statsmodels: %s' % stats.__version__)
print('tensorflow: %s' % tf.__version__)
print('matplotlib: %s' % matplotlib.__version__)
print('scipy: %s' % scipy.__version__)

pandas: 0.18.1
numpy: 1.13.1
tensorflow: 1.2.0
matplotlib: 2.0.0
scipy: 0.18.1


In [4]:
# input the data
amb_temp = pd.read_csv("amb_temp.csv")
comp_temp = pd.read_csv("comp_temp.csv")
mcu_temp = pd.read_csv("mcu_temp.csv")
current = pd.read_csv("current.csv")
power = pd.read_csv("power.csv")
#---
door = pd.read_csv("door.csv")
energy = pd.read_csv("energy.csv")
power_factor = pd.read_csv("power_factor.csv")
voltage = pd.read_csv("voltage.csv")
zcontrol = pd.read_csv("zcontrol.csv")

#mearging the data and a bit cleaning.
data = pd.concat([amb_temp, comp_temp, mcu_temp ,current ,door,energy ,power ,power_factor ,voltage ,zcontrol], axis = 1)
data.columns = ["date",'amb_temp', "date",'comp_temp', "date",'mcu_temp' ,"date",'current' ,"date",'door',"date",'energy' ,"date",'power' ,"date",'power_factor' ,"date",'voltage' ,"date",'zcontrol']
data = data[['amb_temp','comp_temp','mcu_temp' ,'current' ,'power' ]]
#data.fillna(data.mean, inplace=True)
currentarray = np.array(data["current"].values)
amb_temparray = np.array(data["amb_temp"].values)
comp_temparray = np.array(data["comp_temp"].values)
mcu_temparray = np.array(data["mcu_temp"].values)
powerarray = np.array(data["power"].values)

#---
#doorarray = np.array(data["door"].values)
#energyarray = np.array(data["energy"].values)
#power_factorarray = np.array(data["power_factor"].values)
#voltagearray = np.array(data["voltage"].values)
#zcontrolarray = np.array(data["zcontrol"].values)

In [5]:
amb0 = amb_temparray[0:100]
mcu0 = mcu_temparray[0:100]
comp0 = comp_temparray[0:100]
power0 = powerarray[0:100]
current0 = currentarray[0:100]

In [6]:
amb1 = amb_temparray[1:101]
mcu1 = mcu_temparray[1:101]
comp1 = comp_temparray[1:101]
power1 = powerarray[1:101]
current1 = currentarray[1:101]

In [7]:
amb2 = amb_temparray[2:102]
mcu2 = mcu_temparray[2:102]
comp2 = comp_temparray[2:102]
power2 = powerarray[2:102]
current2 = currentarray[2:102]

In [8]:
#PlaceHolders 
amblag0 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "amblag0")
amblag1 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "amblag0")
mculag0 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "amblag0")
mculag1 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "amblag0")
complag0 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "xprev")
complag1 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "xprev")
powerlag0 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "xprev")
powerlag1 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "xprev")
currentlag0 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "xprev")
currentlag1 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "xprev")

amblag2 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "amblag2")
mculag2 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "amblag0")
complag2 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "xprev")
powerlag2 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "xprev")
currentlag2 = tf.placeholder(shape=[100,], dtype=tf.float32, name = "xprev")

# variable for Granger-Lasso coeffiecients 
A1 = tf.Variable(1,dtype=tf.float32)
A2= tf.Variable(1,dtype=tf.float32)
A3 = tf.Variable(1,dtype=tf.float32)
A4 = tf.Variable(1,dtype=tf.float32)
A5 = tf.Variable(1,dtype=tf.float32)
A6 = tf.Variable(1,dtype=tf.float32)
A7 = tf.Variable(1,dtype=tf.float32)
A8 = tf.Variable(1,dtype=tf.float32)
A9 = tf.Variable(1,dtype=tf.float32)
A10 = tf.Variable(1,dtype=tf.float32)

#Output 
one = tf.add(tf.multiply(A1,amblag0),tf.multiply(A2,amblag1))
two = tf.add(tf.multiply(A3,complag0),tf.multiply(A4,complag1))
three = tf.add(tf.multiply(A5,mculag0),tf.multiply(A6,mculag1))
four = tf.add(tf.multiply(A7,currentlag0),tf.multiply(A8,currentlag1))
five = tf.add(tf.multiply(A9,powerlag0),tf.multiply(A10,powerlag1))
next1 = tf.add(one,two)
next2 = tf.add(three,four)
end = tf.add(next1,next2)

model_output = tf.add(end,five)

In [9]:
loss_amb = tf.square(tf.subtract(amblag2,tf.multiply(amblag1,model_output)))

In [10]:
alpha = 0.0001
my_opt = tf.train.GradientDescentOptimizer(alpha)
train_step = my_opt.minimize(loss_amb)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [11]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [12]:
storeamb = []

sess.run(train_step, feed_dict={amblag0 :amb0, amblag1:amb1, mculag0 : mcu0 ,mculag1 : mcu1 ,complag0 :comp0, complag1 :comp1 ,powerlag0 : power0 ,powerlag1 : power1 ,currentlag0 :current0 ,currentlag1 : current1,amblag2 : amb2}) 
print("A1 is", sess.run(A1))
print("A2 is", sess.run(A2))
print("A3 is", sess.run(A3))
print("A4 is", sess.run(A4))
print("A5 is", sess.run(A5))
print("A6 is", sess.run(A6))
print("A7 is", sess.run(A7))
print("A8 is", sess.run(A8))
print("A9 is", sess.run(A9))
print("A10 is", sess.run(A10))
storeamb.append(sess.run(A1))
storeamb.append(sess.run(A2))
storeamb.append(sess.run(A3))
storeamb.append(sess.run(A4))
storeamb.append(sess.run(A5))
storeamb.append(sess.run(A6))
storeamb.append(sess.run(A7))
storeamb.append(sess.run(A8))
storeamb.append(sess.run(A9))
storeamb.append(sess.run(A10))

A1 is -1.08662e+06
A2 is -1.08692e+06
A3 is -1.11825e+06
A4 is -1.17308e+06
A5 is -350708.0
A6 is -346147.0
A7 is -325818.0
A8 is -325870.0
A9 is -7.20694e+07
A10 is -7.20022e+07


In [13]:
storeamb

[-1086615.1,
 -1086924.9,
 -1118251.0,
 -1173084.3,
 -350708.16,
 -346146.91,
 -325817.72,
 -325870.16,
 -72069360.0,
 -72002216.0]

In [14]:
loss_comp = tf.square(tf.subtract(complag2,tf.multiply(complag1,model_output)))

In [15]:
alpha = 0.0001
my_opt = tf.train.GradientDescentOptimizer(alpha)
train_step = my_opt.minimize(loss_comp)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [16]:
storecomp = []

sess.run(train_step, feed_dict={amblag0 :amb0, amblag1:amb1, mculag0 : mcu0 ,mculag1 : mcu1 ,complag0 :comp0, complag1 :comp1 ,powerlag0 : power0 ,powerlag1 : power1 ,currentlag0 :current0 ,currentlag1 : current1, complag2 : comp2}) 
print("A1 is", sess.run(A1))
print("A2 is", sess.run(A2))
print("A3 is", sess.run(A3))
print("A4 is", sess.run(A4))
print("A5 is", sess.run(A5))
print("A6 is", sess.run(A6))
print("A7 is", sess.run(A7))
print("A8 is", sess.run(A8))
print("A9 is", sess.run(A9))
print("A10 is", sess.run(A10))
storecomp.append(sess.run(A1))
storecomp.append(sess.run(A2))
storecomp.append(sess.run(A3))
storecomp.append(sess.run(A4))
storecomp.append(sess.run(A5))
storecomp.append(sess.run(A6))
storecomp.append(sess.run(A7))
storecomp.append(sess.run(A8))
storecomp.append(sess.run(A9))
storecomp.append(sess.run(A10))

A1 is -1.27573e+06
A2 is -1.27608e+06
A3 is -1.33323e+06
A4 is -1.39803e+06
A5 is -411223.0
A6 is -404698.0
A7 is -404531.0
A8 is -381677.0
A9 is -8.9477e+07
A10 is -8.43112e+07


In [17]:
loss_power = tf.square(tf.subtract(powerlag2,tf.multiply(powerlag1,model_output)))

In [18]:
alpha = 0.0001
my_opt = tf.train.GradientDescentOptimizer(alpha)
train_step = my_opt.minimize(loss_power)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [19]:
storepower = []

sess.run(train_step, feed_dict={amblag0 :amb0, amblag1:amb1, mculag0 : mcu0 ,mculag1 : mcu1 ,complag0 :comp0, complag1 :comp1 ,powerlag0 : power0 ,powerlag1 : power1 ,currentlag0 :current0 ,currentlag1 : current1, powerlag2 : power2}) 
print("A1 is", sess.run(A1))
print("A2 is", sess.run(A2))
print("A3 is", sess.run(A3))
print("A4 is", sess.run(A4))
print("A5 is", sess.run(A5))
print("A6 is", sess.run(A6))
print("A7 is", sess.run(A7))
print("A8 is", sess.run(A8))
print("A9 is", sess.run(A9))
print("A10 is", sess.run(A10))
storepower.append(sess.run(A1))
storepower.append(sess.run(A2))
storepower.append(sess.run(A3))
storepower.append(sess.run(A4))
storepower.append(sess.run(A5))
storepower.append(sess.run(A6))
storepower.append(sess.run(A7))
storepower.append(sess.run(A8))
storepower.append(sess.run(A9))
storepower.append(sess.run(A10))

A1 is -5.82196e+09
A2 is -5.82356e+09
A3 is -5.85922e+09
A4 is -6.27658e+09
A5 is -1.90214e+09
A6 is -1.87488e+09
A7 is -1.89602e+09
A8 is -2.1768e+09
A9 is -4.1935e+11
A10 is -4.81533e+11


In [20]:
loss_mcu = tf.square(tf.subtract(mculag2,tf.multiply(mculag1,model_output)))

In [21]:
alpha = 0.0001
my_opt = tf.train.GradientDescentOptimizer(alpha)
train_step = my_opt.minimize(loss_mcu)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [22]:
storemcu = []

sess.run(train_step, feed_dict={amblag0 :amb0, amblag1:amb1, mculag0 : mcu0 ,mculag1 : mcu1 ,complag0 :comp0, complag1 :comp1 ,powerlag0 : power0 ,powerlag1 : power1 ,currentlag0 :current0 ,currentlag1 : current1, mculag2 : mcu2}) 
print("A1 is", sess.run(A1))
print("A2 is", sess.run(A2))
print("A3 is", sess.run(A3))
print("A4 is", sess.run(A4))
print("A5 is", sess.run(A5))
print("A6 is", sess.run(A6))
print("A7 is", sess.run(A7))
print("A8 is", sess.run(A8))
print("A9 is", sess.run(A9))
print("A10 is", sess.run(A10))
storemcu.append(sess.run(A1))
storemcu.append(sess.run(A2))
storemcu.append(sess.run(A3))
storemcu.append(sess.run(A4))
storemcu.append(sess.run(A5))
storemcu.append(sess.run(A6))
storemcu.append(sess.run(A7))
storemcu.append(sess.run(A8))
storemcu.append(sess.run(A9))
storemcu.append(sess.run(A10))

A1 is -110340.0
A2 is -110372.0
A3 is -112797.0
A4 is -118616.0
A5 is -35675.7
A6 is -35235.3
A7 is -32914.1
A8 is -33707.0
A9 is -7.28119e+06
A10 is -7.44896e+06


In [23]:
loss_current = tf.square(tf.subtract(currentlag2,tf.multiply(currentlag1,model_output)))

In [24]:
alpha = 0.0001
my_opt = tf.train.GradientDescentOptimizer(alpha)
train_step = my_opt.minimize(loss_current)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [25]:
storecurrent = []

sess.run(train_step, feed_dict={amblag0 :amb0, amblag1:amb1, mculag0 : mcu0 ,mculag1 : mcu1 ,complag0 :comp0, complag1 :comp1 ,powerlag0 : power0 ,powerlag1 : power1 ,currentlag0 :current0 ,currentlag1 : current1, currentlag2 : current2}) 
print("A1 is", sess.run(A1))
print("A2 is", sess.run(A2))
print("A3 is", sess.run(A3))
print("A4 is", sess.run(A4))
print("A5 is", sess.run(A5))
print("A6 is", sess.run(A6))
print("A7 is", sess.run(A7))
print("A8 is", sess.run(A8))
print("A9 is", sess.run(A9))
print("A10 is", sess.run(A10))
storecurrent.append(sess.run(A1))
storecurrent.append(sess.run(A2))
storecurrent.append(sess.run(A3))
storecurrent.append(sess.run(A4))
storecurrent.append(sess.run(A5))
storecurrent.append(sess.run(A6))
storecurrent.append(sess.run(A7))
storecurrent.append(sess.run(A8))
storecurrent.append(sess.run(A9))
storecurrent.append(sess.run(A10))

A1 is -119179.0
A2 is -119211.0
A3 is -119958.0
A4 is -128464.0
A5 is -38932.3
A6 is -38375.1
A7 is -38748.8
A8 is -44522.1
A9 is -8.57075e+06
A10 is -9.84288e+06


In [22]:
data.describe()

C:\Users\sharan\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,amb_temp,comp_temp,mcu_temp,current,door,energy,power,power_factor,voltage,zcontrol
count,23020.000000,23020.000000,23020.000000,23017.000000,23020.000000,22694.000000,23017.000000,23017.000000,23017.000000,23020.0
mean,29.888196,31.885758,9.059751,6.261266,0.909861,262060.419847,1389.522483,0.933945,246.338398,1.0
std,2.930325,4.706552,0.740041,4.976801,0.286387,77652.080031,1090.411178,0.034697,5.024132,0.0
min,23.812000,19.687000,7.625000,0.819000,0.000000,132556.000000,187.000000,0.840000,232.699997,1.0
25%,27.750000,28.062000,8.562000,NaN,1.000000,NaN,NaN,NaN,NaN,1.0
50%,29.437000,32.000000,8.937000,NaN,1.000000,NaN,NaN,NaN,NaN,1.0
75%,31.687000,35.375000,9.312000,NaN,1.000000,NaN,NaN,NaN,NaN,1.0
max,37.750000,44.062000,14.375000,19.261000,1.000000,399124.000000,2792.000000,0.980000,260.299988,1.0


In [23]:
# I have not used zcontrol because there is only one value in it, hence making it pointless.
data.zcontrol.unique()

array([1], dtype=int64)

In [26]:
matrix = np.matrix([storemcu,storeamb,storecomp, storepower, storecurrent])

In [27]:
referancedata = pd.DataFrame(data = matrix.T)

In [28]:
referancedata.shape

(10, 5)

In [29]:
referancedata.columns = ["mcu_tempref","amb_tempref","comp_tempref","powerref","currentref"]

In [30]:
referancedata

,mcu_tempref,amb_tempref,comp_tempref,powerref,currentref
0,-1.103402e+05,-1.086615e+06,-1.275731e+06,-5.821957e+09,-1.191786e+05
1,-1.103716e+05,-1.086925e+06,-1.276079e+06,-5.823561e+09,-1.192111e+05
2,-1.127968e+05,-1.118251e+06,-1.333230e+06,-5.859223e+09,-1.199581e+05
3,-1.186158e+05,-1.173084e+06,-1.398028e+06,-6.276581e+09,-1.284641e+05
4,-3.567566e+04,-3.507082e+05,-4.112232e+05,-1.902138e+09,-3.893228e+04
5,-3.523527e+04,-3.461469e+05,-4.046979e+05,-1.874879e+09,-3.837513e+04
6,-3.291414e+04,-3.258177e+05,-4.045308e+05,-1.896023e+09,-3.874881e+04
7,-3.370704e+04,-3.258702e+05,-3.816770e+05,-2.176795e+09,-4.452213e+04
8,-7.281194e+06,-7.206936e+07,-8.947702e+07,-4.193502e+11,-8.570752e+06
9,-7.448962e+06,-7.200222e+07,-8.431115e+07,-4.815331e+11,-9.842877e+06


In [39]:
stda 

4.8006097236042615

In [38]:
stda = current0.std()

In [33]:
k = referancedata.values.std(ddof=1)

In [46]:
def calcrefscore(k,series):
    stdb = series.std()
    stda = k
    parta = -(np.log(stda)-np.log(stdb))
    partb = -0.5*(1-np.square(stda)/np.square(stdb))
    return (parta-partb)

In [47]:
calcrefscore(k,current0)

-1.7289424999024286e+20

In [48]:
calcrefscore(k,comp0)

-3.859658210946838e+20

In [49]:
calcrefscore(k,amb0)

-5.9675161192326683e+22

In [50]:
calcrefscore(k,power0)

-3582104813195490.0

In [51]:
calcrefscore(k,mcu0)

-4.0136855071861781e+22